In [15]:
using HDF5, EzXML, Glob, Dates, Plots, Optim, LineSearches, LsqFit, Plots, Optim, LineSearches, LsqFit, JuMP, StatsBase, DSP, BayesOpt
import NLopt
include("/Users/jjc/CSF/CSF Dynamics Scripts/readCSF.jl")

function errfun(Rcsf::Real, E::Real, P_0::Real)
    err = 0.0
    ΔP = Data["P_b"] - P_0
    I_b = ΔP / Rcsf
    It = I_b + Data["I_inf"]
    for i = 1:length(Pm)
        tᵢ = (i - 1) / 6
        Pᵢ = It * ΔP / (I_b + Data["I_inf"] * exp(-E * It * tᵢ)) + P_0 + (Data["I_inf"] * Data["one_needle"])
        err += (Pm[i] - Pᵢ)^2
    end
    global goodness_of_fit = 100 * (sqrt(err) / length(Pm) / abs(mean(Pm)))
    return err
end

function errfunBayes(x)
    Rcsf = x[1]
    E = x[2]
    P_0 = x[3]

    err = 0.0
    ΔP = P_b - P_0
    I_b = ΔP / Rcsf
    It = I_b + I_inf
    for i = 1:length(Pm)
        tᵢ = (i - 1) / 6
        Pᵢ = It * ΔP / (I_b + Data["I_inf"] * exp(-E * It * tᵢ)) + P_0 + (Data["I_inf"] * Data["one_needle"])
        err += (Pm[i] - Pᵢ)^2
    end
    # δlb = delta.(Ib_lower .- I_b)
    # δub = delta.(I_b .- Ib_upper)
    # # δlb = -log.(I_b .- Ib_lower)
    # # δub = -log.(Ib_upper .- I_b)
    # δ = C .* vcat(δlb, δub)
    # penalty = sum(δ .^ κ)
    global goodness_of_fit = 100*(sqrt(err)/length(Pm)/abs(mean(Pm)))
    return err
end

datapath = "/Users/jjc/Documents/SSC/recs_june_2022"
path = pwd();
savepath = "/Users/jjc/CSF/"
files = glob("*.hdf5", datapath);

for f = 1:length(files)
    filename = files[f]
    global Data = readCSF(filename)
    infusion_start_frame = Data["infusion_start_frame"]
    infusion_end_frame = Data["infusion_end_frame"]
    plateau_start = Data["plateau_start"]
    plateau_end = Data["plateau_end"]
    P_b = Data["P_b"]
    P_p = Data["P_p"]
    I_inf = Data["I_inf"]

    try
        global Pm = Data["ICP"][Data["infusion_start_frame"]:Data["infusion_end_frame"]]
    catch
        continue
    end

    # Optimization settings: use JuMP and NLopt
    model = Model(NLopt.Optimizer)
    # set_optimizer_attribute(model, "algorithm", :GN_DIRECT_L_NOSCAL)
    set_optimizer_attribute(model, "algorithm", :GN_DIRECT_L)
    # set_optimizer_attribute(model, "algorithm", :LD_MMA)
    register(model, :errfun, 3, errfun, autodiff=true)
    @variable(model, 0.01 <= Rcsf <= 50.0)
    @variable(model, 0.001 <= E <= 1.0)
    @variable(model, -10.0 <= P_0 <= Data["P_b"])
    # @NLconstraint(model, c1, (P_b - P_0) / Rcsf <= 0.5)
    @NLobjective(model, Min, errfun(Rcsf, E, P_0))

    set_start_value(Rcsf, Data["Rcsf"])
    set_start_value(E, minimum([Data["E"], 1.0]))
    set_start_value(P_0, minimum([0.0, P_b]))
    # set_start_value(P_0, Data["P_0"])

    JuMP.optimize!(model)

    optimize!(model)
    # @show objective_value(model)

    I_b = (Data["P_b"] - value(P_0)) / value(Rcsf)

    plot_model(I_b, value(E), value(P_0), Data["ICP"], 20, false)
    title!("I_b = $I_b\n" * "Rcsf = $(value(Rcsf))\n" * "E = $(value(E))\n" * "P_0 = $(value(P_0)))\n" * "error = $goodness_of_fit")
    savefig("/Users/jjc/Documents/SSC/recs_june_2022/new/" * filename[length(datapath)+2:end-5] * "_New.pdf")
end

In [16]:

function errfun(Rcsf::Real, E::Real, P_0::Real, Pss::Real)
    err = 0.0
    I_b = (Data["P_b"] - Pss) / Rcsf
    It = I_b + Data["I_inf"]
    ΔP = Data["P_b"] - P_0

    for i = 1:length(Pm)
        tᵢ = (i - 1) / 6
        predᵢ = It * ΔP / (I_b + (I_inf * exp(-E * It * tᵢ))) + P_0 + (Data["I_inf"]*Data["one_needle"])
        err += (Data["ICP"][i] - predᵢ)^2
    end
    
    return err
end

function getmodel(optalg, x0, lower, upper)
    model = Model(NLopt.Optimizer) # Initiate instance of model object
    set_optimizer_attribute(model, "algorithm", optalg) # Set optimization algorithm

    register(model, :errfun, 4, errfun, autodiff=true)

    @variable(model, lower[1] <= Rcsf <= upper[1])
    @variable(model, lower[2] <= E <= upper[2])
    @variable(model, lower[3] <= P_0 <= Data["P_b"])
    @variable(model, lower[4] <= Pss <= Data["P_b"])

    @NLobjective(model, Min, errfun(Rcsf, E, P_0, Pss))

    set_start_value(Rcsf, x0[1])
    set_start_value(E, x0[2])
    set_start_value(P_0, x0[3])
    set_start_value(Pss, x0[4])

    JuMP.optimize!(model)

    return value(Rcsf), value(E), value(P_0), value(Pss)
end

optalg = :GN_DIRECT_L_NOSCAL
x0 = [5.0, 0.3, 0.0, 0.0]
lower = [0.01, 0.01, -10.0, -10.0]
upper = [50.0, 1.0, P_b, P_b]
Rcsf, E, P_0, Pss = getmodel(optalg, x0, lower, upper)

I_b = (Data["P_b"] - value(P_0))/value(Rcsf)
plot_model(I_b, value(E), value(P_0), Data["ICP"], 20, false)
title!("I_b = $I_b\n" * "Rcsf = $(value(Rcsf))\n" * "E = $(value(E))\n" * "P_0 = $(value(P_0)))\n" * "Pss = $(value(Pss)))\n" * "error = $goodness_of_fit")

UndefVarError: UndefVarError: P_b not defined

In [12]:
minimum([Data["E"], 1.0])

0.04

In [38]:
using HDF5, EzXML, Glob, Dates, Plots, Optim, LineSearches, LsqFit, Plots, Optim, LineSearches, LsqFit, JuMP, StatsBase, DSP, BayesOpt
import NLopt
include("/Users/jjc/CSF/CSF Dynamics Scripts/readCSF.jl")

datapath = "/Users/jjc/Documents/SSC/recs_june_2022"
path = pwd();
savepath = "/Users/jjc/CSF/"
files = glob("*.hdf5", datapath);

for f = 1:length(files)
    filename = files[f]
    global Data = readCSF(filename)
    infusion_start_frame = Data["infusion_start_frame"]
    infusion_end_frame = Data["infusion_end_frame"]
    plateau_start = Data["plateau_start"]
    plateau_end = Data["plateau_end"]
    P_b = Data["P_b"]
    P_p = Data["P_p"]
    I_inf = Data["I_inf"]

    try
        global Pm = Data["ICP"][Data["infusion_start_frame"]:Data["infusion_end_frame"]]
    catch
        continue
    end

    lowerbound = [0.01, 0.001, -10.0]
    upperbound = [40.0, 1.0, Data["P_b"]]
    global Ib_lower = 0.01
    global Ib_upper = 1.0
    global κ = 2
    global C = 10^3

    # BayesOpt params
    bkernel = "kMaternISO5"
    bsctype = SC_MAP
    bltype = L_MCMC

    # NLOpt params
    optalg = :GN_DIRECT_L

    Rcsf, I_b, E, P_0 = getModelBayes(lowerbound, upperbound, bkernel, bsctype, bltype)
    # Rcsf, I_b, E, P_0 = getModelNL(lowerbound, upperbound, optalg)

    plot_model(I_b, E, P_0, Data["ICP"], 20, false)
    title!("I_b = $I_b\n" * "Rcsf = $(value(Rcsf))\n" * "E = $(value(E))\n" * "P_0 = $(value(P_0)))\n" * "error = $goodness_of_fit")
    savefig("/Users/jjc/Documents/SSC/recs_june_2022/new/" * filename[length(datapath)+2:end-5] * "_New.pdf")
end